In [61]:
from contextlib import contextmanager
import sqlite3
import threading
@contextmanager
def get_connection():
    """Safe connection handler with automatic cleanup"""
    conn = None
    try:
        conn = sqlite3.connect('mydatabase.db', timeout=10)
        conn.execute("PRAGMA busy_timeout = 5000")  # 5 second timeout
        yield conn
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        raise
    finally:
        if conn:
            conn.close()
get_connection()


In [3]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
# edu=enduser/table structure
edutable=""" CREATE TABLE user (
            Userid INTEGER(25) NOT NULL PRIMARY KEY,
            First_Name CHAR(25) NOT NULL,
            Last_Name CHAR(25),
            Status INTEGER(25),
            Email VARCHAR(255),
            Password VARCHAR(255) NOT NULL
            ); """

#creates the table
cursor.execute(edutable)
cursor.close()
conn.close()

In [1]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE user")
conn.close()

In [3]:
import requests

response = requests.get("https://openlibrary.org/search.json?q=*&sort=rating&limit=100")
data = response.json()

In [4]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE book")
conn.close()

In [6]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
bktable=""" CREATE TABLE book (
            Bookid VARCHAR(255) NOT NULL PRIMARY KEY,
            Title CHAR(25) NOT NULL,
            Author CHAR(25),
            Status INTEGER(25),
            Description TEXT,
            Date TEXT,
            Img TEXT,
            Fine INT
            ); """

#creates the table
cursor.execute(bktable)
conn.close()

In [ ]:
import os
APIKEY = os.getenv("API_KEY")
params = {
        "q":data["docs"][0]['title'], 
        "key":APIKEY,
    }
response = requests.get("https://www.googleapis.com/books/v1/volumes",params=params)
response.raise_for_status()
data2 = response.json()


In [12]:
data2['items'][0]['volumeInfo']['imageLinks']['thumbnail']

'http://books.google.com/books/content?id=06ydBAAAQBAJ&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api'

In [ ]:
import requests
import sqlite3
from dotenv import load_dotenv
import os


response = requests.get("https://openlibrary.org/search.json?q=*&sort=rating&limit=100")
data = response.json()
APIKEY = os.getenv("API_KEY")

conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

j=0
for  i in data["docs"]:
    params = {
        "q": i["title"], 
        "key":APIKEY,
    }
    try:
        response = requests.get("https://www.googleapis.com/books/v1/volumes",params=params)
        response.raise_for_status()
        data2 = response.json()
        data3=data2['items'][0]['volumeInfo']
        query = f"""INSERT INTO book 
                    (Bookid, Title, Author, Status, Description, Date,Img,Fine) 
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
        values = (
        f'B{j}', 
        i["title"], 
        data3['authors'][0], 
        0, 
        data3['description'],
        data3['publishedDate'],
        data3['imageLinks']['thumbnail'],
        0
        )
        cursor.execute(query, values)
        conn.commit()
        j=j+1
    except:
        pass
conn.close()
    

In [15]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
btable = """
CREATE TABLE IF NOT EXISTS borrow (
    UserID VARCHAR(255) NOT NULL,
    BookID VARCHAR(255) NOT NULL,
    BorrowDate TEXT NOT NULL DEFAULT CURRENT_DATE,
    DueDate TEXT NOT NULL,
    PRIMARY KEY (UserID, BookID),
    FOREIGN KEY (UserID) REFERENCES user(UserID) ON DELETE CASCADE,
    FOREIGN KEY (BookID) REFERENCES book(BookID) ON DELETE CASCADE
);
"""
cursor.execute(btable)
conn.close()

In [14]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE borrow")
conn.close()

In [2]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
atable = """
CREATE TABLE IF NOT EXISTS author (
            Name CHAR(225) NOT NULL PRIMARY KEY,
            Bdate VARCHAR(225),
            Discription TEXT,
            Bplace TEXT,
            Nationality TEXT,
            Age INT
);
"""
cursor.execute(atable)
conn.close()

In [1]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE author")
conn.close()

In [5]:
import sqlite3
authors_data = [
    ('Jenn Bennett', '1980-08-01', 'An American author known for young adult contemporary and fantasy novels.', 'New York City, New York, USA', 'American', 44),
    ('Brandon Sanderson', '1975-12-19', 'An American fantasy and science fiction writer known for the Cosmere universe and completing Robert Jordan''s The Wheel of Time series.', 'Lincoln, Nebraska, USA', 'American', 49),
    ('Rick Riordan', '1964-06-05', 'An American author known for his Percy Jackson & the Olympians series, which features Greek mythology.', 'San Antonio, Texas, USA', 'American', 61),
    ('Douglas Adams', '1952-03-11', 'An English writer, humorist, and dramatist, best known for The Hitchhiker''s Guide to the Galaxy.', 'Cambridge, England', 'British', 49),
    ('Art Spiegelman', '1948-02-13', 'An American cartoonist, editor, and comics advocate, best known for his Pulitzer Prize-winning graphic novel Maus.', 'Stockholm, Sweden', 'American', 77),
    ('Shannon Messenger', '1982-01-01', 'An American author of middle grade and young adult fantasy novels, known for the Keeper of the Lost Cities series.', 'Southern California, USA', 'American', 43),
    ('Leigh Bardugo', '1975-04-10', 'An Israeli-American fantasy author, best known for her Grishaverse novels.', 'Jerusalem, Israel', 'American', 50),
    ('Dav Pilkey', '1966-03-04', 'An American author and illustrator of children''s books, best known for the Captain Underpants and Dog Man series.', 'Cleveland, Ohio, USA', 'American', 59),
    ('Scott Cawthon', '1971-07-26', 'An American video game designer, developer, and author, best known as the creator of the Five Nights at Freddy''s franchise.', 'Houston, Texas, USA', 'American', 53),
    ('Mo Willems', '1968-02-11', 'An American writer, animator, and children''s book author, known for his humorous picture books like the Elephant & Piggie series.', 'Chicago, Illinois, USA', 'American', 57),
    ('Frederick Barthelme', '1943-10-18', 'An American novelist and short story writer, associated with the literary movement of minimalism.', 'Houston, Texas, USA', 'American', 81),
    ('Terry Pratchett', '1948-04-28', 'An English humorist, satirist, and fantasy writer, best known for his Discworld series.', 'Beaconsfield, England', 'British', 66),
    ('C. J. Howard', None, 'Author of books related to science and technology.', None, None, None),
    ('Neil Lerner', None, 'An academic and author, often associated with musicology or related fields.', None, None, None),
    ('John Ronald Reuel Tolkien', '1892-01-03', 'An English writer, poet, philologist, and university professor, best known as the author of The Hobbit and The Lord of the Rings.', 'Bloemfontein, Orange Free State', 'British', 81),
    ('Lamar Giles', '1979-12-07', 'An American author of young adult novels, focusing on thrillers and mysteries.', 'Hopewell, Virginia, USA', 'American', 45),
    ('Andy Weir', '1972-06-16', 'An American science fiction writer, best known for his debut novel The Martian.', 'Davis, California, USA', 'American', 52),
    ('Ericka Waller', None, 'A contemporary author known for her insightful and often emotionally resonant novels.', None, None, None),
    ('Dave Pelzer', '1960-12-29', 'An American author, best known for his inspirational and autobiographical books, including A Child Called "It".', 'Daly City, California, USA', 'American', 64),
    ('Trevor Noah', '1984-02-20', 'A South African comedian, television host, and author, known for hosting The Daily Show.', 'Johannesburg, South Africa', 'South African', 41),
    ('Orson Scott Card', '1951-08-24', 'An American science fiction writer, known for his novel Ender''s Game.', 'Richland, Washington, USA', 'American', 73),
    ('Dave Gibbons', '1949-04-14', 'An English comic book artist and writer, known for his work on Watchmen.', 'London, England', 'British', 76),
    ('Tobias Wolff', '1945-06-19', 'An American short story writer, memoirist, and novelist, known for his autobiographical works.', 'Ouray, Colorado, USA', 'American', 79),
    ('Raina Telgemeier', '1976-05-26', 'An American cartoonist, known for her graphic novels for young readers, including Smile and Sisters.', 'San Francisco, California, USA', 'American', 49),
    ('Robert Venditti', '1971-10-19', 'An American comic book writer, known for his work on titles such as X-O Manowar and Green Lantern.', 'New York City, New York, USA', 'American', 53),
    ('David Graeber', '1961-02-12', 'An American anthropologist, anarchist activist, and author known for his books on anti-capitalism and social theory.', 'New York City, New York, USA', 'American', 59),
    ('Jeff Kinney', '1971-02-19', 'An American author and cartoonist, best known for the Diary of a Wimpy Kid book series.', 'Fort Washington, Maryland, USA', 'American', 54),
    ('Koyoharu Gotouge', '1989-05-05', 'A Japanese manga artist, known for the hit series Demon Slayer: Kimetsu no Yaiba.', 'Fukuoka Prefecture, Japan', 'Japanese', 36),
    ('Ryoji Hirano', '1986-04-29', 'A Japanese manga artist, known for spin-off works related to popular series like Demon Slayer.', 'Japan', 'Japanese', 39),
    ('Diana Wynne Jones', '1934-08-16', 'An English fantasy writer, known for her intricate and imaginative novels, including Howl''s Moving Castle.', 'London, England', 'British', 76),
    ('George R.R. Martin', '1948-09-20', 'An American novelist and short story writer in the fantasy, horror, and science fiction genres, best known for A Song of Ice and Fire series.', 'Bayonne, New Jersey, USA', 'American', 76),
    ('Khaled Hosseini', '1965-03-04', 'An Afghan-American novelist and physician, known for his novels The Kite Runner and A Thousand Splendid Suns.', 'Kabul, Afghanistan', 'American', 60),
    ('Mary Shelley', '1797-08-30', 'An English novelist, best known for her Gothic novel Frankenstein.', 'London, England', 'British', 53),
    ('Joanne Kathleen Rowling', '1965-07-31', 'A British author, best known by her pen name J. K. Rowling, author of the Harry Potter fantasy series.', 'Yate, England', 'British', 59),
    ('Janell Cannon', '1957-10-27', 'An American children''s author and illustrator, known for her picture books featuring unusual animals.', 'St. Paul, Minnesota, USA', 'American', 67),
    ('J.K. Rowling', '1965-07-31', 'A British author, best known by her pen name J. K. Rowling, author of the Harry Potter fantasy series.', 'Yate, England', 'British', 59),
    ('Ted Chiang', '1967-12-05', 'An American science fiction writer, known for his short stories, many of which have won major awards.', 'Port Jefferson, New York, USA', 'American', 57),
    ('Judith Bauer Stamper', '1953-06-15', 'An American children''s author, known for educational and historical books for young readers.', 'Brooklyn, New York, USA', 'American', 71),
    ('Neil Gaiman', '1960-11-10', 'An English author of short fiction, novels, comic books, graphic novels, nonfiction, audio theatre, and films, known for The Sandman and American Gods.', 'Portchester, England', 'British', 64),
    ('Kristin Hannah', '1960-09-25', 'An American author known for her historical fiction and contemporary novels, often focusing on themes of resilience and relationships.', 'Garden Grove, California, USA', 'American', 64),
    ('Patrick Ness', '1971-10-17', 'An American-British author of young adult fiction, known for the Chaos Walking series and A Monster Calls.', 'Fairfax, Virginia, USA', 'American', 53),
    ('Mary H. K. Choi', '1983-05-24', 'An American author, journalist, and cartoonist, known for her young adult novels.', 'Seoul, South Korea', 'American', 42),
    ('David G. Leahy', None, 'An author associated with academic or specialized non-fiction works.', None, None, None),
    ('Frank Herbert', '1920-10-08', 'An American science fiction author best known for the 1965 novel Dune and its five sequels.', 'Tacoma, Washington, USA', 'American', 65),
    ('Avul Pakir Jainulabdeen Abdul Kalam', '1931-10-15', 'An Indian aerospace scientist and politician who served as the 11th President of India, also a prolific author.', 'Rameswaram, India', 'Indian', 83),
    ('J.R.R. Tolkien', '1892-01-03', 'An English writer, poet, philologist, and university professor, best known as the author of The Hobbit and The Lord of the Rings.', 'Bloemfontein, Orange Free State', 'British', 81),
    ('Michael Cunningham', '1952-11-06', 'An American novelist and short story writer, known for his novel The Hours, which won the Pulitzer Prize.', 'Cincinnati, Ohio, USA', 'American', 72),
    ('R. J. Palacio', '1963-07-13', 'An American author and graphic designer, best known for her novel Wonder.', 'New York City, New York, USA', 'American', 61),
    ('Sergei Lukyanenko', '1968-04-11', 'A Russian science fiction and fantasy writer, known for his Night Watch series.', 'Karagandy, Kazakhstan', 'Russian', 57),
    ('Joseph Maddrey', None, 'An American author and filmmaker, known for his books on film history and true crime.', None, 'American', None),
    ('George Orwell', '1903-06-25', 'An English novelist, essayist, journalist, and critic, best known for his dystopian novel Nineteen Eighty-Four and the allegorical novella Animal Farm.', 'Motihari, British India', 'British', 46),
    ('Octavia E. Butler', '1947-06-22', 'An American science fiction writer, known for her strong female protagonists and exploration of themes like race, sex, and power.', 'Pasadena, California, USA', 'American', 58),
    ('Kazu Kibuishi', '1978-08-08', 'A Japanese-American graphic novelist and illustrator, best known for his Amulet series.', 'Tokyo, Japan', 'American', 46),
    ('Patrick Rothfuss', '1973-06-06', 'An American fantasy writer, best known for his Kingkiller Chronicle series.', 'Madison, Wisconsin, USA', 'American', 51),
    ('George R. R. Martin', '1948-09-20', 'An American novelist and short story writer in the fantasy, horror, and science fiction genres, best known for A Song of Ice and Fire series.', 'Bayonne, New Jersey, USA', 'American', 76),
    ('Antoine de Saint-Exupéry', '1900-06-29', 'A French writer, poet, aristocrat, journalist, and pioneering aviator, best known for his novella The Little Prince.', 'Lyon, France', 'French', 44),
    ('Phil Knight', '1938-02-24', 'An American businessman and philanthropist, co-founder of Nike, Inc., and author of the memoir Shoe Dog.', 'Portland, Oregon, USA', 'American', 87),
    ('William Dalrymple', '1965-03-20', 'A Scottish historian, art historian, and writer, known for his works on Indian history and travel.', 'East Lothian, Scotland', 'Scottish', 60),
    ('Colleen Hoover', '1989-12-11', 'An American author of romance and young adult fiction, known for her emotional and often dramatic novels.', 'Sulphur Springs, Texas, USA', 'American', 35),
    ('Murray Stein', '1943-09-17', 'An American Jungian analyst and author, known for his writings on analytical psychology and Jungian theory.', 'USA', 'American', 81),
    ('Jenny Han', '1990-09-03', 'An American author of young adult fiction, known for The Summer I Turned Pretty and To All the Boys I''ve Loved Before series.', 'Richmond, Virginia, USA', 'American', 34),
    ('Alice Oseman', '1994-10-16', 'An English author and illustrator, best known for her Heartstopper graphic novel series.', 'Chatham, England', 'British', 30)
]
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
insert_sql = """
        INSERT OR IGNORE INTO author (Name, Bdate, Discription, Bplace, Nationality, Age)
        VALUES (?, ?, ?, ?, ?, ?)
    """
cursor.executemany(insert_sql, authors_data)
conn.commit()
conn.close()


In [ ]:
import requests
from datetime import datetime

def get_author_details(author_name):
    # SPARQL Query
    query = f"""
    SELECT ?author ?authorLabel ?birthDate ?deathDate ?image ?birthPlaceLabel ?nationalityLabel
    WHERE {{
      ?author wdt:P31 wd:Q5;
              rdfs:label "{author_name}"@en;
              wdt:P569 ?birthDate.
      OPTIONAL {{ ?author wdt:P570 ?deathDate. }}
      OPTIONAL {{ ?author wdt:P18 ?image. }}
      OPTIONAL {{ ?author wdt:P19 ?birthPlace. }}
      OPTIONAL {{ ?author wdt:P27 ?nationality. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    
    # API Request
    url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/json"}
    response = requests.get(url, params={"query": query}, headers=headers,timeout=10)
    data = response.json()
    
    # Process Results
    if not data.get("results", {}).get("bindings"):
        return {"error": "Author not found"}
    
    # Extract unique data (avoid duplicates)
    unique_data = {}
    for item in data["results"]["bindings"]:
        author_id = item["author"]["value"].split("/")[-1]  # Extract QID
        
        if author_id not in unique_data:
            unique_data[author_id] = {
                "name": item["authorLabel"]["value"],
                "birth_date": item["birthDate"]["value"][:10],  # YYYY-MM-DD
                "death_date": item.get("deathDate", {}).get("value", "")[:10],
                "images": set(),
                "birth_place": item.get("birthPlaceLabel", {}).get("value"),
                "nationalities": set()
            }
        
        # Add images and nationalities (avoid duplicates)
        if "image" in item:
            unique_data[author_id]["images"].add(item["image"]["value"])
        if "nationalityLabel" in item:
            unique_data[author_id]["nationalities"].add(item["nationalityLabel"]["value"])
    
    # Convert sets to lists
    for author in unique_data.values():
        author["images"] = list(author["images"])
        author["nationalities"] = list(author["nationalities"])
        
        # Calculate age
        birth_year = int(author["birth_date"][:4])
        if author["death_date"]:
            age = int(author["death_date"][:4]) - birth_year
        else:
            age = datetime.now().year - birth_year
        author["age"] = age
    
    return list(unique_data.values())[0]  # Return first author




In [88]:
import time
import tableaction
table=tableaction.Tableaction()
import requests
import sqlite3
from dotenv import load_dotenv
import os
import requests
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
data=table.getauth()
for  i in data:
    try:
        author_data = get_author_details(i)
        img = author_data.get('images', [])  # Returns empty list if 'images' doesn't exist
        img = img[0] if len(img) > 0 else "nan"
        
        # Safely handle nationalities
        nationality = author_data.get('nationalities', [])
        nationality = nationality[0] if len(nationality) > 0 else "Unknown"
        query = f"""INSERT INTO author 
                        (Name, Bdate, Image, Bplace, Nationality, Age) 
                        VALUES (?, ?, ?, ?, ?, ?)"""
        values = (
        author_data['name'],
        author_data['birth_date'],
        img,
        author_data['birth_place'],
        nationality,
        author_data['age']
        )
        cursor.execute(query, values)
        conn.commit()
    except Exception as e:
        print(i)
        print("An error occurred:", e)
        print(author_data)
    time.sleep(0.2)
conn.close()        


Jenn Bennett
An error occurred: 'name'
{'error': "API request failed: HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=10)"}


KeyboardInterrupt: 

In [47]:
len(data)

62